In [ ]:
import pandas as pd
# Charger le fichier CSV
df = pd.read_csv("fichier_lyon.csv")

# Sélectionner les colonnes pertinentes
df_dc = df[['siren', 'codePostalEtablissement', 'dateCreationEtablissement']]
df_dc.head()

In [ ]:
# Remplacer les 'NN' par NaN
df_dc = df_dc.replace('NN', pd.NA)

# Supprimer les lignes contenant des NaN
df_dc = df_dc.dropna()

# Convertir la colonne 'dateCreationEtablissement' en datetime
df_dc['dateCreationEtablissement'] = pd.to_datetime(df_dc['dateCreationEtablissement'])

df_dc.head()

In [ ]:
# Dictionnaire de mappage pour les arrondissements
arrondissement_mapping = {
    '69001': 'Lyon 1',
    '69002': 'Lyon 2',
    '69003': 'Lyon 3',
    '69004': 'Lyon 4',
    '69005': 'Lyon 5',
    '69006': 'Lyon 6',
    '69007': 'Lyon 7',
    '69008': 'Lyon 8',
    '69009': 'Lyon 9'
}

# Créer une nouvelle colonne 'arrondissement' avec les noms d'arrondissements
df_dc['arrondissement'] = df_dc['codePostalEtablissement'].astype(str).map(arrondissement_mapping)

# Réorganiser les colonnes pour que 'arrondissement' soit en troisième position
cols = ['siren', 'codePostalEtablissement', 'arrondissement', 'dateCreationEtablissement']
df_dc = df_dc[cols]

# Ajouter "FR-" devant chaque code postal
df_dc['codePostalEtablissement'] = df_dc['codePostalEtablissement'].apply(lambda x: 'FR-' + str(x))

# Afficher les informations et les premières lignes pour vérifier
df_dc.info()
df_dc.head()

In [5]:
df_dc.to_csv('fichier_lyon_dc.csv', index=False)

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as cx

# Charger les données CSV
df_entreprises = pd.read_csv('fichier_lyon_dc.csv')

# Compter le nombre d'entreprises par arrondissement
df_count = df_entreprises.groupby('arrondissement').size().reset_index(name='nombre_entreprises')

# Charger les données géographiques (par exemple un GeoJSON des arrondissements de Lyon)
gdf_arrondissements = gpd.read_file('metropole-de-lyon_adr_voie_lieu.adrarrond.json')

# Reprojeter en EPSG:3857 pour la visualisation sur la carte
gdf_lyon_web_mercator = gdf_arrondissements.to_crs(epsg=3857)

# Fusionner les deux DataFrames sur le nom de l'arrondissement
gdf_merged = gdf_arrondissements.merge(df_count, left_on='nomreduit', right_on='arrondissement')

# Créer une carte avec une coloration en fonction du nombre d'entreprises
gdf_merged.plot(column='nombre_entreprises', cmap='Blues', legend=True)

# Créer une figure et un axe pour la carte
fig, ax = plt.subplots(figsize=(10, 8))

# Ajouter le fond de carte
cx.add_basemap(ax, source=cx.providers.CartoDB.Positron)

# Titre et mise en page
plt.title('concentration des entreprises à Lyon', fontsize=16)
plt.axis('off')

# Récupérer les limites actuelles de la carte
minx, miny, maxx, maxy = gdf_lyon_web_mercator.total_bounds

# Facteur de zoom (plus grand que 1 pour zoomer)
zoom_factor = 1  # Modifie cette valeur pour plus de zoom (1.2 signifie zoom arrière, 0.8 pour zoom avant)

# Calculer les nouvelles limites avec le zoom
x_range = maxx - minx
y_range = maxy - miny

new_minx = minx - (zoom_factor - 1) * x_range / 2
new_maxx = maxx + (zoom_factor - 1) * x_range / 2
new_miny = miny - (zoom_factor - 1) * y_range / 2
new_maxy = maxy + (zoom_factor - 1) * y_range / 2

# Ajuster les limites de la carte avec le zoom
ax.set_xlim([new_minx, new_maxx])
ax.set_ylim([new_miny, new_maxy])

# Afficher la carte
plt.tight_layout()
plt.show()